In [31]:
import h5py
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from plotly.subplots import make_subplots
import time

target=r"J:\ctgroup\Edward\DATA\VMI\20250307\po4w_angle_sweep\15.cv4"
t_range=[0,10_000]

xy_hist=np.zeros((256,256))
xy_bins=np.linspace(0,256,257)
xy_c=np.linspace(0,255,256)
t_hist=np.zeros(1024)
t_bins=np.linspace(t_range[0],t_range[1],1025)
t_c=np.linspace(t_range[0],t_range[1],1024)


df=pd.DataFrame(columns=["x","y","t","shot_index"])
current_idx=0

fig=make_subplots(rows=1,cols=2,subplot_titles=("XY","T"))
fig.update_layout(width=1200,height=600)
fig=go.FigureWidget(fig)
fig.add_trace(go.Heatmap(z=xy_hist),row=1,col=1)
fig.add_trace(go.Bar(x=t_c,y=t_hist),row=1,col=2)
display(fig)


def load_data(file,curr_idx,df=None,l=-1):
    if l==-1:
        l=len(file["x"])-curr_idx
    x=file["x"][curr_idx:curr_idx+l]
    y=file["y"][curr_idx:curr_idx+l]
    t=file["t"][curr_idx:curr_idx+l]
    pulse=file["cluster_corr"][curr_idx:curr_idx+l]
    df=pd.concat([df,pd.DataFrame({"x":x,"y":y,"t":t, "shot_index":pulse})]) if df is not None else pd.DataFrame({"x":x,"y":y,"t":t, "shot_index":pulse})
    return df

def update_hist(x,y,t,xy_hist,t_hist):
    xy_hist+=np.histogram2d(x,y,bins=[xy_bins,xy_bins])[0]
    t_hist+=np.histogram(t,bins=t_bins)[0]
    return xy_hist,t_hist

def update_plot(f,xy_hist,t_hist):
    f.data[0].z=xy_hist/np.sum(xy_hist)
    f.data[1].y=t_hist/np.sum(t_hist)
    
    
file=h5py.File(target,"r")
print(file.keys())
def update():
    global current_idx,df,xy_hist,t_hist,file
    df=load_data(file,current_idx,l=100000)
    df=df[df["t"]<300]
    xy_hist,t_hist=update_hist(df["x"],df["y"],df["t"],xy_hist,t_hist)
    update_plot(fig,xy_hist,t_hist)
    fig.update_layout(title=f"Shots: {df['shot_index'].iloc[-1]}, Counts: {int(sum(t_hist))}, Count Rate: {sum(t_hist)/df['shot_index'].iloc[-1]:.2f}")
    current_idx+=len(df)
    
#update every 1 second
for i in range(30):
    update()
    time.sleep(1)
    

FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': '520d5cc7-2bd7-486a-b4f7-cc4590ef6dba',
              'xaxis': 'x',
              'yaxis': 'y',
              'z': array([[0., 0., 0., ..., 0., 0., 0.],
                          [0., 0., 0., ..., 0., 0., 0.],
                          [0., 0., 0., ..., 0., 0., 0.],
                          ...,
                          [0., 0., 0., ..., 0., 0., 0.],
                          [0., 0., 0., ..., 0., 0., 0.],
                          [0., 0., 0., ..., 0., 0., 0.]])},
             {'type': 'bar',
              'uid': '4b316cc5-20d2-4263-8529-6842e09e2fb4',
              'x': array([0.00000000e+00, 9.77517107e+00, 1.95503421e+01, ..., 9.98044966e+03,
                          9.99022483e+03, 1.00000000e+04]),
              'xaxis': 'x2',
              'y': array([0., 0., 0., ..., 0., 0., 0.]),
              'yaxis': 'y2'}],
    'layout': {'annotations': [{'font': {'size': 16},
                                'showarrow'

<KeysViewHDF5 ['cluster_corr', 'etof_corr', 't', 't_etof', 't_pulse', 't_tof', 'tof_corr', 'x', 'y']>
